In [8]:
 # Importing data actu stops  
import fiona
import geopandas
actu_stops = geopandas.read_file("0-data/2109_STIB_MIVB_Network/ACTU_STOPS.dbf")

# saving csv 
actu_stops.to_csv('all_stops.csv', index=False)


In [68]:
# Function convertions coordinates
def convertCoords(df):
    x2,y2 = transform(inProj,outProj,df['coord_x'],df['coord_y'])
    return pd.Series({'newLong':x2,'newLat':y2})

In [69]:
df[['newLong','newLat']] = df.apply(convertCoords,axis=1)

<ipython-input-68-607b37ea429a>:3: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj,outProj,df['coord_x'],df['coord_y'])
/Users/errazioussama/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [70]:
df.head()

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry,newLong,newLat
2987,001m,1,1,8733,GARE DE L'OUEST,WESTSTATION,Gare de l'Ouest,Weststation,146633.5,170956.4,M,1,STOCKEL,POINT (146633.500 170956.400),1.317231,1.535544
2988,001m,1,2,8742,BEEKKANT,BEEKKANT,Beekkant,Beekkant,146776.5,171444.3,M,1,STOCKEL,POINT (146776.500 171444.300),1.318516,1.539925
2989,001m,1,3,8292,ETANGS NOIRS,ZWARTE VIJVERS,Étangs Noirs,Zwarte Vijvers,147492.7,171859.9,M,1,STOCKEL,POINT (147492.700 171859.900),1.324949,1.543657
2990,001m,1,4,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,Comte de Flandre,Graaf van Vlaanderen,148013.6,171590.4,M,1,STOCKEL,POINT (148013.600 171590.400),1.329629,1.541237
2991,001m,1,5,8272,SAINTE-CATHERINE,SINT-KATELIJNE,Sainte-Catherine,Sint-Katelijne,148539.5,171278.2,M,1,STOCKEL,POINT (148539.500 171278.200),1.334353,1.538433


In [43]:
# Importing the geodesic module from the library
from geopy.distance import geodesic
  
# Loading the lat-long data for Kolkata & Delhi
coords_1  = (1.317231, 1.535544)
coords_2 = (1.318516, 1.539925)
  
# Print the distance calculated in meters 
print(geodesic(coords_1, coords_2).m)

507.84489627878935


In [72]:
# vectorized haversine function to compute the distance (Reference:  https://stackoverflow.com/questions/40452759/pandas-latitude-longitude-to-distance-between-successive-rows)
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


# Derive the distance
df['dist'] = \
    haversine(df.newLat.shift(), df.newLong.shift(),
                 df.loc[1:, 'newLat'], df.loc[1:, 'newLong'])


/Users/errazioussama/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [73]:
df

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry,newLong,newLat,dist
2987,001m,1,1,8733,GARE DE L'OUEST,WESTSTATION,Gare de l'Ouest,Weststation,146633.5,170956.4,M,1,STOCKEL,POINT (146633.500 170956.400),1.317231,1.535544,NaN
2988,001m,1,2,8742,BEEKKANT,BEEKKANT,Beekkant,Beekkant,146776.5,171444.3,M,1,STOCKEL,POINT (146776.500 171444.300),1.318516,1.539925,0.507673
2989,001m,1,3,8292,ETANGS NOIRS,ZWARTE VIJVERS,Étangs Noirs,Zwarte Vijvers,147492.7,171859.9,M,1,STOCKEL,POINT (147492.700 171859.900),1.324949,1.543657,0.826823
2990,001m,1,4,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,Comte de Flandre,Graaf van Vlaanderen,148013.6,171590.4,M,1,STOCKEL,POINT (148013.600 171590.400),1.329629,1.541237,0.585619
2991,001m,1,5,8272,SAINTE-CATHERINE,SINT-KATELIJNE,Sainte-Catherine,Sint-Katelijne,148539.5,171278.2,M,1,STOCKEL,POINT (148539.500 171278.200),1.334353,1.538433,0.610683
2992,001m,1,6,8012,DE BROUCKERE,DE BROUCKERE,De Brouckère,De Brouckère,148854.5,171046.9,M,1,STOCKEL,POINT (148854.500 171046.900),1.337183,1.536356,0.390222
2993,001m,1,7,8022,GARE CENTRALE,CENTRAAL STATION,Gare Centrale,Centraal Station,149297.4,170660.6,M,1,STOCKEL,POINT (149297.400 170660.600),1.341161,1.532887,0.586829
2994,001m,1,8,8032,PARC,PARK,Parc,Park,149593.4,170587.0,M,1,STOCKEL,POINT (149593.400 170587.000),1.343820,1.532226,0.304563
2995,001m,1,9,8042,ARTS-LOI,KUNST-WET,Arts-Loi,Kunst-Wet,150061.1,170547.6,M,1,STOCKEL,POINT (150061.100 170547.600),1.348022,1.531873,0.468664
2996,001m,1,10,8052,MAELBEEK,MAALBEEK,Maelbeek,Maalbeek,150622.3,170360.8,M,1,STOCKEL,POINT (150622.300 170360.800),1.353063,1.530195,0.590600
